In [1]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
from transformers import BertTokenizerFast, AlbertModel, BertModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import hanja
from imblearn.over_sampling import RandomOverSampler
from transformers.optimization import get_cosine_schedule_with_warmup, get_linear_schedule_with_warmup
from transformers import AdamW
from collections import defaultdict

In [13]:
PATH = '../Data/'

In [14]:
train = pd.read_csv(PATH + 'train_감정분류.csv', index_col=0)
test = pd.read_csv(PATH + 'test_감정분류.csv', index_col=0)

In [15]:
emotion = list(train['감정_소분류'].unique())

emotion_ls = []
for t in train['감정_소분류']:
    emotion_ls.append(emotion.index(t))
    
train['감정_대분류'] = emotion_ls

In [16]:
emotion = list(test['감정_소분류'].unique())

emotion_ls = []
for t in test['감정_소분류']:
    emotion_ls.append(emotion.index(t))
    
test['감정_대분류'] = emotion_ls

In [17]:
x_train = train['응답정리']
y_train = train['감정_대분류']

In [18]:
y_train

0         0
1         0
2         0
3         0
4         0
         ..
228463    1
228464    1
228465    1
228466    1
228467    1
Name: 감정_대분류, Length: 228468, dtype: int64

In [19]:
x_test = test['응답정리']
y_test = test['감정_대분류']

---

In [20]:
tokenizer_bert_kor_base = BertTokenizerFast.from_pretrained("kykim/albert-kor-base")

Downloading: 100%|██████████| 81.0/81.0 [00:00<00:00, 42.3kB/s]
Downloading: 100%|██████████| 336k/336k [00:00<00:00, 375kB/s]  
Downloading: 100%|██████████| 684/684 [00:00<00:00, 319kB/s]
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizerFast'. 
The class this function is called from is 'BertTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizerFast'. 
The class this function is called from is 'BertTokenizerFast'.


In [21]:
titles_t = x_train.to_numpy().reshape(-1,1)
labels_t = y_train.to_numpy().reshape(-1,1)
# oversample = RandomOverSampler()
# X_over, y_over = oversample.fit_resample(titles_t, labels_t)
# train = pd.DataFrame({'title':X_over.reshape(-1), 'topic_idx':y_over.reshape(-1)})
train = pd.DataFrame({'title':titles_t.reshape(-1), 'topic_idx':labels_t.reshape(-1)})

In [22]:
titles_t = x_test.to_numpy().reshape(-1,1)
labels_t = y_test.to_numpy().reshape(-1,1)
# oversample = RandomOverSampler()
# X_over, y_over = oversample.fit_resample(titles_t, labels_t)
# train = pd.DataFrame({'title':X_over.reshape(-1), 'topic_idx':y_over.reshape(-1)})
test = pd.DataFrame({'title':titles_t.reshape(-1), 'topic_idx':labels_t.reshape(-1)})

In [26]:
# 뉴스 데이터 클래스 생성
class NewsSubjectDataset(Dataset):
  def __init__(self, subjects, targets, tokenizer, max_len):
    self.subjects = subjects
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len
  def __len__(self):
    return len(self.subjects)
  def __getitem__(self, item):
    subject = str(self.subjects[item])
    target = self.targets[item]
    encoding = self.tokenizer.encode_plus(
      subject,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      padding = 'max_length',
      truncation = True,
      return_attention_mask=True,
      return_tensors='pt',
    )
    return {
      'subject_text': subject,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }
def create_data_loader(df, tokenizer, max_len, batch_size, shuffle_=False, valid=False):
  if valid:
    ds = NewsSubjectDataset(
      subjects=df.title.to_numpy(),
      targets=np.zeros(len(df)),
      tokenizer=tokenizer,
      max_len=max_len
      )
  else:
    ds = NewsSubjectDataset(
      subjects=df.title.to_numpy(),
      targets=df.topic_idx.to_numpy(),
      tokenizer=tokenizer,
      max_len=max_len
    )
  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=4,
    shuffle = shuffle_
  )
  
# 데이터 로더 생성
BATCH_SIZE =64
MAX_LEN =32
train_data_loader = create_data_loader(train, tokenizer_bert_kor_base, MAX_LEN, BATCH_SIZE, shuffle_=True)
valid_data_loader = create_data_loader(test, tokenizer_bert_kor_base, MAX_LEN, BATCH_SIZE, valid=True)
import random
class NewsSubjectClassifier(nn.Module):
  def __init__(self, n_classes):
    super(NewsSubjectClassifier, self).__init__()
    self.bert = AlbertModel.from_pretrained("kykim/albert-kor-base")
    self.drop = nn.Dropout(p=0.5)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  def forward(self, input_ids, attention_mask):
    _, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask,
       return_dict=False
    )
    output = self.drop(pooled_output)
    return self.out(output)

device = torch.device("cpu")
def get_predictions(model, data_loader):
  model = model.eval()
  subject_texts = []
  predictions = []
  prediction_probs = []
  with torch.no_grad():
    for d in data_loader:
      texts = d["subject_text"]
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)
      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)
      subject_texts.extend(texts)
      predictions.extend(preds)
      prediction_probs.extend(outputs)
  predictions = torch.stack(predictions).cpu()
  prediction_probs = torch.stack(prediction_probs).cpu()
  return subject_texts, predictions, prediction_probs
import gc
EPOCHS = 2

In [27]:
model_albert_kor_base = NewsSubjectClassifier(n_classes=58).to(device)
optimizer = AdamW(model_albert_kor_base.parameters(), lr=1e-4)
total_steps = len(train_data_loader) * EPOCHS
## cosine_scheduler or linear,  warmup or no warmup
scheduler = get_cosine_schedule_with_warmup(
  optimizer,
  num_warmup_steps=int(total_steps*0.1),
  num_training_steps=total_steps
)
loss_fn = nn.CrossEntropyLoss().to(device)

from tqdm import tqdm
def train_epoch(model,data_loader,loss_fn,optimizer,device,scheduler,n_examples):
  model = model.train()
  losses = []
  correct_predictions = 0
  for d in tqdm(data_loader):
    input_ids = d["input_ids"].to(device)
    attention_mask = d["attention_mask"].to(device)
    targets = d["targets"].to(device)
    outputs = model(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, targets)
    correct_predictions += torch.sum(preds == targets)
    losses.append(loss.item())
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
  return correct_predictions.double() / n_examples, np.mean(losses)

Some weights of the model checkpoint at kykim/albert-kor-base were not used when initializing AlbertModel: ['predictions.bias', 'predictions.decoder.weight', 'predictions.dense.weight', 'predictions.LayerNorm.weight', 'sop_classifier.classifier.bias', 'predictions.dense.bias', 'predictions.decoder.bias', 'sop_classifier.classifier.weight', 'predictions.LayerNorm.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [28]:
for epoch in range(EPOCHS):
  print(f'Epoch {epoch + 1}/{EPOCHS}')
  print('-' * 10)
  train_acc, train_loss = train_epoch(
    model_albert_kor_base,
    train_data_loader,
    loss_fn,
    optimizer,
    device,
    scheduler,
    len(train)
  )
  print(f'Train loss {train_loss} accuracy {train_acc}')

y_review_texts, y_pred, y_pred_probs = get_predictions(
  model_albert_kor_base,
  valid_data_loader
)
answers = []
answers.append(y_pred.tolist())
gc.collect()
torch.cuda.empty_cache()

Epoch 1/2
----------


  0%|          | 0/3570 [00:00<?, ?it/s]Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/homebrew/Caskroom/miniforge/base/envs/multi_crawling/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/opt/homebrew/Caskroom/miniforge/base/envs/multi_crawling/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'NewsSubjectDataset' on <module '__main__' (built-in)>
  0%|          | 0/3570 [01:49<?, ?it/s]


KeyboardInterrupt: 